In [ ]:
# Extract fires between 2018-2022
daily_ = daily[(daily['ig_year'] >= 2018) & (daily['ig_year'] <= 2022)]
print(f"There are {len(daily_['id'].unique())} unique fires.\n\t[{len(daily_)}] individual daily perimeters.")

In [ ]:
# Identify classes with 'aspen' in the EVT_NAME
aspen_evt = props_df_event[props_df_event['EVT_NAME'].str.contains('aspen', case=False, na=False)]['EVT_NAME'].unique()
print(aspen_evt)

In [ ]:
# Now filter to retain wildfire events with at least 5% aspen forest cover
props_filtered = props_df[props_df['EVT_NAME'].isin(aspen_evt)]
aspen_sum = props_filtered.groupby('fired_id')['pct_cover'].sum().reset_index() # get the sum of aspen classes
aspen_sum['pct_aspen'] = aspen_sum['pct_cover']  # rename the column to retain aspen percent

# Filter out fires with less than 5% aspen cover
aspen_fires = aspen_sum[aspen_sum['pct_aspen'] >= 5] # retain fires with >= 5% 
aspen_fires = aspen_fires[['fired_id','pct_aspen']] # subset columns
print(aspen_fires)

In [ ]:
# Join back to the FIRED data
# Check for matching fired_id set
common_ids = set(fired['fired_id']).intersection(set(aspen_fires['fired_id'])) # Find intersection
print(f"Number of common IDs: {len(common_ids)}")

# Join the attribute data to FIRED polygons
fired['fired_id'] = fired['fired_id'].astype(str)
aspen_fires['fired_id'] = aspen_fires['fired_id'].astype(str)

# Join aspen percent to the FIRED data
fired_aspen = pd.merge(fired, aspen_fires, on='fired_id', how='inner')
print(fired_aspen.columns)

In [ ]:
# Filter fires that are only one MODIS pixel
fired_aspen = fired_aspen[fired_aspen['tot_pix'] > 1]
print(fired_aspen['tot_pix'].describe())
# Filter out any "cropland" fires
fired_aspen = fired_aspen[fired_aspen['lc_name'] != 'Croplands']
print(fired_aspen['lc_name'].unique())

print(f'There are {len(fired_aspen)} fire events meeting our criteria so far.')

In [ ]:
# Save this file out
fired_aspen = fired_aspen.to_crs(proj)  # ensure the correct projection before exporting
fired_aspen.to_file(os.path.join(maindir,'aspen-fire/Aim2/data/spatial/mod/FIRED/fired_events_west_aspen_all_gt5pct.gpkg'))

In [ ]:
# Map of aspen fires

In [ ]:
# Spatial map of aspen wildfires (centroid)

# Load the state boundaries
states = gpd.read_file(os.path.join(maindir,'data/boundaries/political/TIGER/tl19_us_states_west_nad83.gpkg'))

# Generate centroids
centroid = fired_aspen.copy()
centroid['geometry'] = centroid.geometry.centroid

# Make a spatial map of the centroids now
fig, ax = plt.subplots(figsize=(6, 6))

states.plot(ax=ax, edgecolor='black', linewidth=1, color='none')

# Plot centroids
centroid['size'] = centroid['pct_aspen'] * 10  # Adjust the scaling factor as necessary
centroid.plot(
    ax=ax, markersize=centroid['pct_aspen'], 
    column='pct_aspen', cmap='viridis', 
    legend=True, alpha=0.6, 
    legend_kwds={'label': "Aspen Percent"})

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)

del centroid

# Save the map as a PNG
plt.savefig(os.path.join(projdir, 'figures/Fig1_all_aspen_fires.png'), dpi=300, bbox_inches='tight')

plt.show()